In [11]:
#imports
import pandas as pd
import numpy as np
import ipaddress

In [12]:
#merging all the csv's
df = pd.concat(
    map(pd.read_csv, 
        ['dataset/Bot-IoT/UNSW_2018_IoT_Botnet_Full5pc_1.csv',
         'dataset/Bot-IoT/UNSW_2018_IoT_Botnet_Full5pc_2.csv',
         'dataset/Bot-IoT/UNSW_2018_IoT_Botnet_Full5pc_3.csv',
         'dataset/Bot-IoT/UNSW_2018_IoT_Botnet_Full5pc_4.csv']), 
    ignore_index=True)

/tmp/ipykernel_7391/3643078548.py:2: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(
/tmp/ipykernel_7391/3643078548.py:2: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(
/tmp/ipykernel_7391/3643078548.py:2: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(
/tmp/ipykernel_7391/3643078548.py:2: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(


In [13]:
df.head()

,pkSeqID,stime,flgs,flgs_number,proto,proto_number,saddr,sport,daddr,dport,...,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,category,subcategory
0,1,1.528089e+09,e,1,tcp,1,192.168.100.147,49960,192.168.100.7,80,...,1.127040,96,75,1.133720,1.129970,770,602,1,DoS,HTTP
1,2,1.528089e+09,e,1,arp,2,192.168.100.7,-1,192.168.100.147,-1,...,15267.200000,1,2,0.005142,0.005142,2,6,1,DoS,HTTP
2,3,1.528089e+09,e,1,tcp,1,192.168.100.147,49962,192.168.100.7,80,...,1.127040,96,75,1.135100,1.129970,770,602,1,DoS,HTTP
3,4,1.528089e+09,e,1,tcp,1,192.168.100.147,49964,192.168.100.7,80,...,1.127040,96,75,1.135140,1.129970,770,602,1,DoS,HTTP
4,5,1.528089e+09,e,1,tcp,1,192.168.100.147,49966,192.168.100.7,80,...,1.127040,96,75,1.135260,1.129970,770,602,1,DoS,HTTP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3668517,3668518,1.529381e+09,e,1,tcp,1,192.168.100.150,35064,192.168.100.3,22,...,9.889330,19,19,455.754000,9.889330,30,30,1,Theft,Keylogging
3668518,3668519,1.529381e+09,e,1,tcp,1,192.168.100.150,35066,192.168.100.3,22,...,9.889330,19,19,10453.000000,9.889330,30,30,1,Theft,Keylogging
3668519,3668520,1.529381e+09,e,1,tcp,1,192.168.100.150,35070,192.168.100.3,22,...,9.889330,19,19,10.785200,9.889330,441,441,1,Theft,Keylogging
3668520,3668521,1.529381e+09,e,1,tcp,1,192.168.100.3,43001,192.168.100.150,4433,...,666667.000000,1,3,666667.000000,22346.400000,2,4,1,Theft,Keylogging


In [15]:
#typecasting into proper format
df['stime'] = df['stime'].astype('int64') 
df['ltime'] = df['ltime'].astype('int64') 

In [16]:
#taking important features
df = df[['spkts','dpkts','sbytes','dbytes','dur','proto','attack','stime','ltime','sport','dport','saddr','daddr','category']]

In [17]:
df.head()

,spkts,dpkts,sbytes,dbytes,dur,proto,attack,stime,ltime,sport,dport,saddr,daddr,category
0,5,3,650,1330,7.056393,tcp,1,1528088521,1528088528,49960,80,192.168.100.147,192.168.100.7,DoS
1,1,1,60,60,0.000131,arp,1,1528088521,1528088521,-1,-1,192.168.100.7,192.168.100.147,DoS
2,5,3,796,1330,7.047852,tcp,1,1528088521,1528088528,49962,80,192.168.100.147,192.168.100.7,DoS
3,5,3,694,1330,7.047592,tcp,1,1528088521,1528088528,49964,80,192.168.100.147,192.168.100.7,DoS
4,5,3,989,1330,7.046841,tcp,1,1528088521,1528088528,49966,80,192.168.100.147,192.168.100.7,DoS


In [18]:
#converting ip address into intgers
df['saddr'] =  df['saddr'].apply(lambda x:int(ipaddress.ip_address(x)))
df['daddr'] =  df['daddr'].apply(lambda x:int(ipaddress.ip_address(x)))

/tmp/ipykernel_7391/664784687.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['saddr'] =  df['saddr'].apply(lambda x:int(ipaddress.ip_address(x)))
/tmp/ipykernel_7391/664784687.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['daddr'] =  df['daddr'].apply(lambda x:int(ipaddress.ip_address(x)))


In [19]:
df.head()

,spkts,dpkts,sbytes,dbytes,dur,proto,attack,stime,ltime,sport,dport,saddr,daddr,category
0,5,3,650,1330,7.056393,tcp,1,1528088521,1528088528,49960,80,3232261267,3232261127,DoS
1,1,1,60,60,0.000131,arp,1,1528088521,1528088521,-1,-1,3232261127,3232261267,DoS
2,5,3,796,1330,7.047852,tcp,1,1528088521,1528088528,49962,80,3232261267,3232261127,DoS
3,5,3,694,1330,7.047592,tcp,1,1528088521,1528088528,49964,80,3232261267,3232261127,DoS
4,5,3,989,1330,7.046841,tcp,1,1528088521,1528088528,49966,80,3232261267,3232261127,DoS


In [20]:
df['proto'].value_counts()

udp          1996437
tcp          1662812
icmp            9052
arp              213
ipv6-icmp          8
Name: proto, dtype: int64

In [21]:
(1996437+1662812)/3668522 

0.9974722790268125

In [22]:
#we only take tcp and udp as by removing other columns we are only rejecting 1% of the data
df = df[(df['proto']=='udp') | (df['proto']=='tcp')]

In [23]:
#replacing udp with 0 and tcp with 1
df['proto'] = np.where(df['proto'] == 'udp', 0, 1)

In [24]:
df['category'].value_counts()

DDoS              1926573
DoS               1650197
Reconnaissance      81982
Normal                422
Theft                  75
Name: category, dtype: int64

In [25]:
#we only need  attack category of dos and ddos type
df = df[(df['category']=='DDoS') | (df['category']=='DoS')]

In [26]:
#resetting index
df.reset_index(drop=True,inplace=True)

In [27]:
df

,spkts,dpkts,sbytes,dbytes,dur,proto,attack,stime,ltime,sport,dport,saddr,daddr,category
0,5,3,650,1330,7.056393,1,1,1528088521,1528088528,49960,80,3232261267,3232261127,DoS
1,5,3,796,1330,7.047852,1,1,1528088521,1528088528,49962,80,3232261267,3232261127,DoS
2,5,3,694,1330,7.047592,1,1,1528088521,1528088528,49964,80,3232261267,3232261127,DoS
3,5,3,989,1330,7.046841,1,1,1528088521,1528088528,49966,80,3232261267,3232261127,DoS
4,5,3,653,1330,7.046596,1,1,1528088521,1528088528,49968,80,3232261267,3232261127,DoS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3576765,8,0,480,0,14.869558,0,1,1528099368,1528099383,13250,80,3232261269,3232261123,DDoS
3576766,8,0,480,0,14.869558,0,1,1528099368,1528099383,13251,80,3232261269,3232261123,DDoS
3576767,8,0,480,0,14.869558,0,1,1528099368,1528099383,13252,80,3232261269,3232261123,DDoS
3576768,8,0,480,0,14.869558,0,1,1528099368,1528099383,13253,80,3232261269,3232261123,DDoS


In [28]:
#splitting data of category Dos into train and test
df_dos = df[df['category'] == 'DoS']
df_dos_train = df_dos.iloc[:1600000]
df_dos_test = df_dos.iloc[1641197:]

In [29]:
#saving csv ino train and test
df_dos_train.to_csv('dataset/cleaned_dataset/train/bot_iot_dos.csv',index=False)
df_dos_test.to_csv('dataset/cleaned_dataset/test/bot_iot_dos.csv',index=False)

In [30]:
#splitting data of category DDos into train and test
df_ddos = df[df['category'] == 'DDoS']
df_ddos.reset_index(drop= True,inplace = True)
df_ddos_train = df_ddos.iloc[:1600000]
df_ddos_test = df_ddos.iloc[1917572:]

In [34]:
#saving csv ino train and test
df_ddos_train.to_csv('dataset/cleaned_dataset/train/bot_iot_ddos.csv',index=False)
df_ddos_test.to_csv('dataset/cleaned_dataset/test/bot_iot_ddos.csv',index=False)